In [16]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric
from torch_geometric.data import Data,DataLoader,InMemoryDataset
import os
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
node_csv = pd.read_csv('train_90.csv')
edge_csv = pd.read_csv('edge_90.csv')

In [5]:
import hashlib
def md5_to_int(md5_string):
    # 使用 hashlib 计算 MD5 哈希值
    md5_hash = hashlib.md5(md5_string.encode()).hexdigest()
    # 将哈希值转换为整数（使用 int() 函数）
    int_value = int(md5_hash, 16)
    return int_value


In [7]:
geohash_id = node_csv.iloc[:,0]
geohash1 = edge_csv.iloc[:,0]
geohash2 = edge_csv.iloc[:,1]

node_date_id = node_csv.iloc[:,1]
edge_date_id = edge_csv.iloc[:,4]
# 从节点数据中提取特征，假设特征列从第三列开始（索引2）
node_features = torch.FloatTensor(node_csv.iloc[:, 2:].values)

# 创建边的索引（两列分别表示连接的节点）Note that edge_index, 
# i.e. the tensor defining the source and target nodes of all edges
# is not a list of index tuples. If you want to write your indices this way
#  you should transpose and call contiguous on it before passing them to the data constructor:
edge_index = torch.Tensor([
    [md5_to_int(geohash1.iloc[i]), md5_to_int(geohash2.iloc[i])]
    for i in range(len(geohash1))
]).t().contiguous()
node_id = torch.Tensor([md5_to_int(geohash_id.iloc[i]) for i in range(len(geohash_id))])
# 创建边的特征
edge_features = torch.FloatTensor(edge_csv.iloc[:, 2:4].values)


In [8]:
class config:
    learning_rate = 0.001
    K = 1 # number of aggregation loop (also means number of GCN layers)
    gcn_agg = 'mean' # aggregator function: mean, conv, lstm, pooling
    filter_noise = True
    seed = 1234
    noise_threshold = 1
    batch_size = 16

GCNConv, CNN, LSTM, SGC, GNN, GAT, GraphSAGE, GIN，Bi-LSTM

In [25]:
class MyDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        # Return a list of raw file names if needed
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def process(self):
        # 创建一个 PyTorch Geometric Data 对象
        data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

        # 添加目标数据（活跃指数和消费指数），假设它们存储在 nodes_data 中的最后两列
        #y (torch.Tensor, optional) – Graph-level or node-level ground-truth labels with arbitrary shape. (default: None)
        target = torch.FloatTensor(node_csv.iloc[:, -2:].values)
        data.y = target

        # 添加时间信息到数据
        data.date_ids = node_date_id.values.T # 时间信息作为节点特征的一部分
        data.edge_dates = edge_date_id.values.T  # 时间信息作为边特征的一部分
        data_list = [data]
        # Save the list of Data objects
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        print(data.num_features)
        print(data.num_nodes)
        print(data.num_edges)


In [26]:
dataset = MyDataset(root='.', transform=None, pre_transform=None)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

37
102600
1048575


Processing...
Done!
d:\Anaconda\Lib\site-packages\torch\utils\data\dataset.py:414: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
